<a href="https://colab.research.google.com/github/navneetkrc/Deep_learning_experiments/blob/master/GLINER_with_best_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio gliner pyngrok transformers streamlit pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.0 MB/s eta 0:00:00


In [2]:
from gliner import GLiNER

model = GLiNER.from_pretrained("knowledgator/gliner-multitask-v1.0")

text = """
Microsoft was founded by Bill Gates and Paul Allen on April 4, 1975 to develop and sell BASIC interpreters for the Altair 8800. During his career at Microsoft, Gates held the positions of chairman, chief executive officer, president and chief software architect, while also being the largest individual shareholder until May 2014.
"""

labels = ["founder", "computer", "software", "position", "date"]

entities = model.predict_entities(text, labels)

for entity in entities:
    print(entity["text"], "=>", entity["label"])


Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Bill Gates => founder
Paul Allen => founder
April 4, 1975 => date
BASIC interpreters => software
Altair 8800 => computer
chairman => position
chief executive officer => position
president => position
chief software architect => position
May 2014 => date


In [3]:
text = """
Microsoft was founded by Bill Gates and Paul Allen on April 4, 1975 to develop and sell BASIC interpreters for the Altair 8800. During his career at Microsoft, Gates held the positions of chairman, chief executive officer, president and chief software architect, while also being the largest individual shareholder until May 2014.
"""

labels = ["Microsoft <> founder", "Microsoft <> inception date", "Bill Gates <> held position"]

entities = model.predict_entities(text, labels)

for entity in entities:
    print(entity["label"], "=>", entity["text"])


Microsoft <> founder => Bill Gates
Microsoft <> founder => Paul Allen
Microsoft <> inception date => April 4, 1975
Bill Gates <> held position => chairman
Bill Gates <> held position => chief executive officer
Bill Gates <> held position => president
Bill Gates <> held position => chief software architect


In [2]:
from google.colab import userdata
userdata.get('NGROK_AUTH_TOKEN')

'2qRLX76ycIV3fRoWMCCmimd2OME_2AFxAeCBpmzskB9L5ZZhs'

In [ ]:
from typing import Dict, Union
from gliner import GLiNER
import gradio as gr
import torch  # Import torch
from pyngrok import ngrok  # Import ngrok
import os

# Use CUDA if available, otherwise use CPU
device = "cuda" if torch.cuda.is_available() else "cpu"
model = GLiNER.from_pretrained("knowledgator/gliner-multitask-v1.0").to(device)

text1 = """
"I recently purchased the Sony WH-1000XM4 Wireless Noise-Canceling Headphones from Amazon and I must say, I'm thoroughly impressed. The package arrived in New York within 2 days, thanks to Amazon Prime's expedited shipping.

The headphones themselves are remarkable. The noise-canceling feature works like a charm in the bustling city environment, and the 30-hour battery life means I don't have to charge them every day. Connecting them to my Samsung Galaxy S21 was a breeze, and the sound quality is second to none.

I also appreciated the customer service from Amazon when I had a question about the warranty. They responded within an hour and provided all the information I needed.

However, the headphones did not come with a hard case, which was listed in the product description. I contacted Amazon, and they offered a 10% discount on my next purchase as an apology.

Overall, I'd give these headphones a 4.5/5 rating and highly recommend them to anyone looking for top-notch quality in both product and service."""


text3 = """
Several studies have reported its pharmacological activities, including anti-inflammatory, antimicrobial, and antitumoral effects.
The effect of E-anethole was studied in the osteosarcoma MG-63 cell line, and the antiproliferative activity was evaluated by an MTT assay.
It showed a GI50 value of 60.25 μM with apoptosis induction through the mitochondrial-mediated pathway. Additionally, it induced cell cycle arrest at the G0/G1 phase, up-regulated the expression of p53, caspase-3, and caspase-9, and down-regulated Bcl-xL expression.
Moreover, the antitumoral activity of anethole was assessed against oral tumor Ca9-22 cells, and the cytotoxic effects were evaluated by MTT and LDH assays.
It demonstrated a LD50 value of 8 μM, and cellular proliferation was 42.7% and 5.2% at anethole concentrations of 3 μM and 30 μM, respectively.
It was reported that it could selectively and in a dose-dependent manner decrease cell proliferation and induce apoptosis, as well as induce autophagy, decrease ROS production, and increase glutathione activity. The cytotoxic effect was mediated through NF-kB, MAP kinases, Wnt, caspase-3 and -9, and PARP1 pathways. Additionally, treatment with anethole inhibited cyclin D1 oncogene expression, increased cyclin-dependent kinase inhibitor p21WAF1, up-regulated p53 expression, and inhibited the EMT markers.
"""

text5 = """
Dr. Paul Hammond, a renowned neurologist at Johns Hopkins University, has recently published a paper in the prestigious journal "Nature Neuroscience". His research focuses on a rare genetic mutation, found in less than 0.01% of the population, that appears to prevent the development of Alzheimer's disease. Collaborating with researchers at the University of California, San Francisco, the team is now working to understand the mechanism by which this mutation confers its protective effect. Funded by the National Institutes of Health, their research could potentially open new avenues for Alzheimer's treatment.
"""

ner_examples = [

    [
        text5,
        "neurologist, scientist, gene, disease, biological process, city, journal, university",
        0.4,
        True
    ],
    [
        text1,
        "product, brand, location, features, rating",
        0.5,
        False
    ],
    [
        text3,
        "cell line, protein, metric, substance",
        0.5,
        False
    ],
    [
        """
* Data Scientist, Data Analyst, or Data Engineer with 1+ years of experience.
* Experience with technologies such as Docker, Kubernetes, or Kubeflow
* Machine Learning experience preferred
* Experience with programming languages such as Python, C++, or SQL preferred
* Experience with technologies such as Databricks, Qlik, TensorFlow, PyTorch, Python, Dash, Pandas, or NumPy preferred
* BA or BS degree
* Active Secret OR Active Top Secret or Active TS/SCI clearance
""",
        "software package, programing language, software tool, degree, job title",
        0.3,
        False,
    ],
    [
        "However, both models lack other frequent DM symptoms including the fibre-type dependent atrophy, myotonia, cataract and male-infertility.",
        "disease, symptom",
        0.3,
        False,
    ],
    [
        "Synergy between signal transduction pathways is obligatory for expression of c-fos in B and T cell lines: implication for c-fos control via surface immunoglobulin and T cell antigen receptors.",
        "DNA, RNA, cell line, cell type, protein",
        0.3,
        False,
    ],
    [
        "The choice of the encoder and decoder modules of dnpg can be quite flexible, for instance long short term memory networks (lstm) or convolutional neural network (cnn).",
        "short acronym, long acronym",
        0.3,
        False,
    ],
    [
        "Amelia Earhart flew her single engine Lockheed Vega 5B across the Atlantic to Paris.",
        "person, company, location, airplane",
        0.3,
        True,
    ],
    [
        "Feldman is a contributor to NBC Sports Boston's ``State of the Revs`` and ``Revolution Postgame Live`` programs as well as to 98.5 the SportsHub, SiriusXM FC's MLS coverage and to other New England and national radio outlets and podcasts.",
        "person, company, location",
        0.3,
        False,
    ],
    [
        "On 25 July 1948, on the 39th anniversary of Bleriot's crossing of the English Channel, the Type 618 Nene-Viking flew Heathrow to Paris (Villacoublay) in the morning carrying letters to Bleriot's widow and son (secretary of the FAI), who met it at the airport.",
        "date, location, person, organization",
        0.3,
        False,
    ],
    [
        "Leo & Ian won the 1962 Bathurst Six Hour Classic at Mount Panorama driving a Daimler SP250 sports car, (that year the 500 mile race for touring cars were held at Phillip Island)",
        "person, date, location, organization, competition",
        0.3,
        False,
    ],
    [
        "The Shore Line route of the CNS & M until 1955 served, from south to north, the Illinois communities of Chicago, Evanston, Wilmette, Kenilworth, Winnetka, Glencoe, Highland Park, Highwood, Fort Sheridan, Lake Forest, Lake Bluff, North Chicago, Waukegan, Zion, and Winthrop Harbor as well as Kenosha, Racine, and Milwaukee (the ``KRM'') in Wisconsin.",
        "location, organization, date",
        0.3,
        False,
    ],
    [
        "Comet C/2006 M4 (SWAN) is a non-periodic comet discovered in late June 2006 by Robert D. Matson of Irvine, California and Michael Mattiazzo of Adelaide, South Australia in publicly available images of the Solar and Heliospheric Observatory (SOHO).",
        "person, organization, date, location",
        0.3,
        False,
    ]]

def merge_entities(entities):
    if not entities:
        return []
    merged = []
    current = entities[0]
    for next_entity in entities[1:]:
        if next_entity['entity'] == current['entity'] and (next_entity['start'] == current['end'] + 1 or next_entity['start'] == current['end']):
            current['word'] += ' ' + next_entity['word']
            current['end'] = next_entity['end']
        else:
            merged.append(current)
            current = next_entity
    merged.append(current)
    return merged

def process(
    text, labels: str, threshold: float, nested_ner: bool
) -> Dict[str, Union[str, int, float]]:
    labels = [label.strip() for label in labels.split(",")]
    r = {
        "text": text,
        "entities": [
            {
                "entity": entity["label"],
                "word": entity["text"],
                "start": entity["start"],
                "end": entity["end"],
                "score": 0,
            }
            for entity in model.predict_entities(
                text, labels, flat_ner=not nested_ner, threshold=threshold
            )
        ],
    }
    r["entities"] =  merge_entities(r["entities"])
    return r


# Set up the Gradio interface
with gr.Blocks(title="NER Task") as ner_interface:
    input_text = gr.Textbox(label="Text input", placeholder="Enter your text here")
    labels = gr.Textbox(label="Labels", placeholder="Enter your labels here (comma separated)", scale=2)
    threshold = gr.Slider(0, 1, value=0.3, step=0.01, label="Threshold", info="Lower the threshold to increase how many entities get predicted.")
    nested_ner = gr.Checkbox(label="Nested NER", info="Allow for nested NER?")
    output = gr.HighlightedText(label="Predicted Entities")
    submit_btn = gr.Button("Submit")
    examples = gr.Examples(
    ner_examples,
    fn=process,
    inputs=[input_text, labels, threshold, nested_ner],
    outputs=output,
    cache_examples=True
    )
    theme=gr.themes.Base()

    input_text.submit(fn=process, inputs=[input_text, labels, threshold, nested_ner], outputs=output)
    labels.submit(fn=process, inputs=[input_text, labels, threshold, nested_ner], outputs=output)
    threshold.release(fn=process, inputs=[input_text, labels, threshold, nested_ner], outputs=output)
    submit_btn.click(fn=process, inputs=[input_text, labels, threshold, nested_ner], outputs=output)
    nested_ner.change(fn=process, inputs=[input_text, labels, threshold, nested_ner], outputs=output)


# Run the app with ngrok
if __name__ == "__main__":
    # Get an ngrok authtoken and set it (replace "YOUR_AUTHTOKEN" with your actual token)
    # You can get your authtoken from https://dashboard.ngrok.com/get-started/your-authtoken
    ngrok_token = os.environ.get("NGROK_AUTH_TOKEN")

    if ngrok_token:
        ngrok.set_auth_token(ngrok_token)

        # Open a tunnel on port 7860 (Gradio's default port)
        public_url = ngrok.connect(7860)
        print(f"Public URL: {public_url}")
        ner_interface.launch(server_port=7860)

    else:
        print("Please set the NGROK_AUTH_TOKEN environment variable with your ngrok authtoken.")
        print("You can get your authtoken from https://dashboard.ngrok.com/get-started/your-authtoken")
        # Fallback to local launch if no token provided (for local testing)
        ner_interface.launch()

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

In [19]:
# Save this code in a file named "ner_app.py"
%%writefile ner_app.py
import streamlit as st
from typing import Dict, Union
from gliner import GLiNER
from pyngrok import ngrok
import pandas as pd
from google.colab import userdata

# Set page configuration
st.set_page_config(page_title="NER App", layout="wide")

# Function to merge adjacent entities of the same type
def merge_entities(entities):
    if not entities:
        return []
    merged = []
    current = entities[0]
    for next_entity in entities[1:]:
        if next_entity['entity'] == current['entity'] and (next_entity['start'] == current['end'] + 1 or next_entity['start'] == current['end']):
            current['word'] += ' ' + next_entity['word']
            current['end'] = next_entity['end']
        else:
            merged.append(current)
            current = next_entity
    merged.append(current)
    return merged

# Load the model
@st.cache_resource
def load_model():
    return GLiNER.from_pretrained("knowledgator/gliner-multitask-v1.0").to("cpu")

# Process text through the NER model
def process_text(text, labels, threshold, nested_ner):
    model = load_model()
    labels = [label.strip() for label in labels.split(",")]

    entities = [
        {
            "entity": entity["label"],
            "word": entity["text"],
            "start": entity["start"],
            "end": entity["end"],
            "score": entity["score"],
        }
        for entity in model.predict_entities(
            text, labels, flat_ner=not nested_ner, threshold=threshold
        )
    ]

    return merge_entities(entities)

# Sample text for example
sample_text = """I recently purchased the Sony WH-1000XM4 Wireless Noise-Canceling Headphones from Amazon and I must say, I'm thoroughly impressed. The package arrived in New York within 2 days, thanks to Amazon Prime's expedited shipping.

The headphones themselves are remarkable. The noise-canceling feature works like a charm in the bustling city environment, and the 30-hour battery life means I don't have to charge them every day. Connecting them to my Samsung Galaxy S21 was a breeze, and the sound quality is second to none.

I also appreciated the customer service from Amazon when I had a question about the warranty. They responded within an hour and provided all the information I needed.

However, the headphones did not come with a hard case, which was listed in the product description. I contacted Amazon, and they offered a 10% discount on my next purchase as an apology.

Overall, I'd give these headphones a 4.5/5 rating and highly recommend them to anyone looking for top-notch quality in both product and service."""

sample_labels = "product, brand, location, features, rating"

# Main app function
def main():
    st.title("Named Entity Recognition App")
    st.write("Identify custom entities in your text using the GLiNER model")

    # Sidebar for parameters
    st.sidebar.header("Model Parameters")
    threshold = st.sidebar.slider("Confidence Threshold", 0.0, 1.0, 0.3, 0.01)
    nested_ner = st.sidebar.checkbox("Enable Nested NER", value=False)

    # Main content
    text_input = st.text_area("Enter your text:", height=200, value=sample_text)
    labels_input = st.text_input("Entity types (comma separated):", value=sample_labels)

    if st.button("Identify Entities"):
        if not text_input or not labels_input:
            st.error("Please enter both text and entity types")
            return

        with st.spinner("Processing..."):
            entities = process_text(text_input, labels_input, threshold, nested_ner)

            if not entities:
                st.info("No entities found with current settings. Try lowering the threshold.")
            else:
                # Display highlighted text
                st.subheader("Identified Entities")
                html_output = text_input

                # Insert highlights from end to beginning
                for entity in sorted(entities, key=lambda x: x['start'], reverse=True):
                    start = entity['start']
                    end = entity['end']
                    entity_type = entity['entity']
                    entity_text = entity['word']

                    # Generate color based on entity type
                    color_hash = hash(entity_type) % 360
                    bgcolor = f"hsl({color_hash}, 70%, 80%)"

                    highlight_html = f'<mark style="background-color: {bgcolor};" title="{entity_type}">{entity_text}</mark>'
                    html_output = html_output[:start] + highlight_html + html_output[end:]

                st.markdown(html_output, unsafe_allow_html=True)

                # Display entities table
                st.subheader("Entity Details")
                entity_df = pd.DataFrame(entities)
                if 'score' in entity_df.columns:
                    entity_df['score'] = entity_df['score'].round(3)
                st.dataframe(entity_df)

# For running in Colab with pyngrok
def run_with_ngrok():
    # Set up ngrok
    ngrok.set_auth_token(userdata.get('NGROK_AUTH_TOKEN'))

    # Start ngrok tunnel
    public_url = ngrok.connect(port=8501)
    st.success(f"App running at: {public_url}")

if __name__ == "__main__":
    # Check if running in Colab
    import os
    is_colab = 'COLAB_GPU' in os.environ

    if is_colab:
        run_with_ngrok()

    main()

Overwriting ner_app.py


In [25]:
# Save this code in a file named "ner_app.py"
%%writefile ner_app.py
# Save this code in a file named "ner_app.py"
import streamlit as st
from typing import Dict, Union
from gliner import GLiNER
from pyngrok import ngrok
import pandas as pd
import os

# Set page configuration
st.set_page_config(page_title="NER App", layout="wide")

# Function to merge adjacent entities of the same type
def merge_entities(entities):
    if not entities:
        return []
    merged = []
    current = entities[0]
    for next_entity in entities[1:]:
        if next_entity['entity'] == current['entity'] and (next_entity['start'] == current['end'] + 1 or next_entity['start'] == current['end']):
            current['word'] += ' ' + next_entity['word']
            current['end'] = next_entity['end']
        else:
            merged.append(current)
            current = next_entity
    merged.append(current)
    return merged

# Load the model
@st.cache_resource
def load_model():
    return GLiNER.from_pretrained("knowledgator/gliner-multitask-v1.0").to("cpu")

# Process text through the NER model
def process_text(text, labels, threshold, nested_ner):
    model = load_model()
    labels = [label.strip() for label in labels.split(",")]

    entities = [
        {
            "entity": entity["label"],
            "word": entity["text"],
            "start": entity["start"],
            "end": entity["end"],
            "score": entity["score"],
        }
        for entity in model.predict_entities(
            text, labels, flat_ner=not nested_ner, threshold=threshold
        )
    ]

    return merge_entities(entities)

# Sample text for example
sample_text = """I recently purchased the Sony WH-1000XM4 Wireless Noise-Canceling Headphones from Amazon and I must say, I'm thoroughly impressed. The package arrived in New York within 2 days, thanks to Amazon Prime's expedited shipping.

The headphones themselves are remarkable. The noise-canceling feature works like a charm in the bustling city environment, and the 30-hour battery life means I don't have to charge them every day. Connecting them to my Samsung Galaxy S21 was a breeze, and the sound quality is second to none.

I also appreciated the customer service from Amazon when I had a question about the warranty. They responded within an hour and provided all the information I needed.

However, the headphones did not come with a hard case, which was listed in the product description. I contacted Amazon, and they offered a 10% discount on my next purchase as an apology.

Overall, I'd give these headphones a 4.5/5 rating and highly recommend them to anyone looking for top-notch quality in both product and service."""

sample_labels = "product, brand, location, features, rating"

# Main app function
def main():
    st.title("Named Entity Recognition App")
    st.write("Identify custom entities in your text using the GLiNER model")

    # Sidebar for parameters
    st.sidebar.header("Model Parameters")
    threshold = st.sidebar.slider("Confidence Threshold", 0.0, 1.0, 0.3, 0.01)
    nested_ner = st.sidebar.checkbox("Enable Nested NER", value=False)

    # Main content
    text_input = st.text_area("Enter your text:", height=200, value=sample_text)
    labels_input = st.text_input("Entity types (comma separated):", value=sample_labels)

    if st.button("Identify Entities"):
        if not text_input or not labels_input:
            st.error("Please enter both text and entity types")
            return

        with st.spinner("Processing..."):
            entities = process_text(text_input, labels_input, threshold, nested_ner)

            if not entities:
                st.info("No entities found with current settings. Try lowering the threshold.")
            else:
                # Display highlighted text
                st.subheader("Identified Entities")
                html_output = text_input

                # Insert highlights from end to beginning
                for entity in sorted(entities, key=lambda x: x['start'], reverse=True):
                    start = entity['start']
                    end = entity['end']
                    entity_type = entity['entity']
                    entity_text = entity['word']

                    # Generate color based on entity type
                    color_hash = hash(entity_type) % 360
                    bgcolor = f"hsl({color_hash}, 70%, 80%)"

                    highlight_html = f'<mark style="background-color: {bgcolor};" title="{entity_type}">{entity_text}</mark>'
                    html_output = html_output[:start] + highlight_html + html_output[end:]

                st.markdown(html_output, unsafe_allow_html=True)

                # Display entities table
                st.subheader("Entity Details")
                entity_df = pd.DataFrame(entities)
                if 'score' in entity_df.columns:
                    entity_df['score'] = entity_df['score'].round(3)
                st.dataframe(entity_df)

# For running in Colab with pyngrok
def run_with_ngrok():
    # Get ngrok auth token from environment variable
    ngrok_token = os.environ.get('NGROK_AUTH_TOKEN')

    if ngrok_token:
        # Set up ngrok with the token
        ngrok.set_auth_token(ngrok_token)

        # Start ngrok tunnel
        public_url = ngrok.connect(port=8501)
        st.success(f"App running at: {public_url}")
        print(f"Public URL: {public_url}")
    else:
        st.error("NGROK_AUTH_TOKEN environment variable not found. Public URL will not be available.")
        print("Error: NGROK_AUTH_TOKEN environment variable not found")

if __name__ == "__main__":
    # Check if running in Colab
    import os
    is_colab = 'COLAB_GPU' in os.environ

    if is_colab:
        run_with_ngrok()

    main()

Overwriting ner_app.py


In [29]:
%%writefile ner_app2.py
# Save this code in a file named "ner_app.py"
import streamlit as st
from typing import Dict, Union
from gliner import GLiNER
from pyngrok import ngrok
import pandas as pd
import os
import re

# Set page configuration
st.set_page_config(page_title="NER App", layout="wide")

# Function to merge adjacent entities of the same type
def merge_entities(entities):
    if not entities:
        return []
    merged = []
    current = entities[0]
    for next_entity in entities[1:]:
        if next_entity['entity'] == current['entity'] and (next_entity['start'] == current['end'] + 1 or next_entity['start'] == current['end']):
            current['word'] += ' ' + next_entity['word']
            current['end'] = next_entity['end']
        else:
            merged.append(current)
            current = next_entity
    merged.append(current)
    return merged

# Load the model
@st.cache_resource
def load_model():
    return GLiNER.from_pretrained("knowledgator/gliner-multitask-v1.0").to("cpu")

# Process text through the NER model
def process_text(text, labels, threshold, nested_ner):
    model = load_model()
    labels = [label.strip() for label in labels.split(",")]

    entities = [
        {
            "entity": entity["label"],
            "word": entity["text"],
            "start": entity["start"],
            "end": entity["end"],
            "score": entity["score"],
        }
        for entity in model.predict_entities(
            text, labels, flat_ner=not nested_ner, threshold=threshold
        )
    ]

    return merge_entities(entities)

# Improved function to safely highlight entities in text
def highlight_entities(text, entities):
    # Sort entities by start position in descending order to avoid offset issues
    sorted_entities = sorted(entities, key=lambda x: x['start'], reverse=True)

    # Create a list of characters from the text
    chars = list(text)

    # Process each entity
    for entity in sorted_entities:
        start = entity['start']
        end = entity['end']
        entity_type = entity['entity']

        # Generate color based on entity type
        color_hash = hash(entity_type) % 360
        bgcolor = f"hsl({color_hash}, 70%, 80%)"

        # Create HTML tags for the entity
        open_tag = f'<span style="background-color: {bgcolor}; padding: 1px 3px; border-radius: 3px;" title="{entity_type}">'
        close_tag = '</span>'

        # Insert the tags
        chars.insert(end, close_tag)
        chars.insert(start, open_tag)

    # Join the characters back into a string and replace newlines with <br> tags
    highlighted_text = ''.join(chars).replace('\n', '<br>')

    # Wrap in a div with appropriate styling
    return f'<div style="line-height: 1.6; white-space: pre-wrap; font-family: sans-serif;">{highlighted_text}</div>'

# Sample text for example
sample_text = """I recently purchased the Sony WH-1000XM4 Wireless Noise-Canceling Headphones from Amazon and I must say, I'm thoroughly impressed. The package arrived in New York within 2 days, thanks to Amazon Prime's expedited shipping.

The headphones themselves are remarkable. The noise-canceling feature works like a charm in the bustling city environment, and the 30-hour battery life means I don't have to charge them every day. Connecting them to my Samsung Galaxy S21 was a breeze, and the sound quality is second to none.

I also appreciated the customer service from Amazon when I had a question about the warranty. They responded within an hour and provided all the information I needed.

However, the headphones did not come with a hard case, which was listed in the product description. I contacted Amazon, and they offered a 10% discount on my next purchase as an apology.

Overall, I'd give these headphones a 4.5/5 rating and highly recommend them to anyone looking for top-notch quality in both product and service."""

sample_labels = "product, brand, location, features, rating"

# Main app function
def main():
    st.title("Named Entity Recognition App")
    st.write("Identify custom entities in your text using the GLiNER model")

    # Sidebar for parameters
    st.sidebar.header("Model Parameters")
    threshold = st.sidebar.slider("Confidence Threshold", 0.0, 1.0, 0.3, 0.01)
    nested_ner = st.sidebar.checkbox("Enable Nested NER", value=False)

    # Main content
    text_input = st.text_area("Enter your text:", height=200, value=sample_text)
    labels_input = st.text_input("Entity types (comma separated):", value=sample_labels)

    if st.button("Identify Entities"):
        if not text_input or not labels_input:
            st.error("Please enter both text and entity types")
            return

        with st.spinner("Processing..."):
            entities = process_text(text_input, labels_input, threshold, nested_ner)

            if not entities:
                st.info("No entities found with current settings. Try lowering the threshold.")
            else:
                # Display highlighted text
                st.subheader("Identified Entities")

                # Use the improved highlighting function
                html_output = highlight_entities(text_input, entities)
                st.markdown(html_output, unsafe_allow_html=True)

                # Display entities table
                st.subheader("Entity Details")
                entity_df = pd.DataFrame(entities)
                if 'score' in entity_df.columns:
                    entity_df['score'] = entity_df['score'].round(3)
                st.dataframe(entity_df)

                # Download options
                st.subheader("Download Results")

                # CSV download button for entity data
                csv = entity_df.to_csv(index=False)
                st.download_button(
                    label="Download Entities as CSV",
                    data=csv,
                    file_name="entities.csv",
                    mime="text/csv",
                )

                # Plain text download with annotations
                annotated_text = text_input
                for entity in sorted(entities, key=lambda x: x['start'], reverse=True):
                    start = entity['start']
                    end = entity['end']
                    entity_type = entity['entity']
                    annotated_text = annotated_text[:start] + f"[{annotated_text[start:end]}]({entity_type})" + annotated_text[end:]

                st.download_button(
                    label="Download Annotated Text",
                    data=annotated_text,
                    file_name="annotated_text.txt",
                    mime="text/plain",
                )

# For running in Colab with pyngrok
def run_with_ngrok():
    # Get ngrok auth token from environment variable
    ngrok_token = os.environ.get('NGROK_AUTH_TOKEN')

    if ngrok_token:
        # Set up ngrok with the token
        ngrok.set_auth_token(ngrok_token)

        try:
            # Start ngrok tunnel
            public_url = ngrok.connect(port=8501)
            st.success(f"App running at: {public_url}")
            print(f"Public URL: {public_url}")
        except Exception as e:
            st.error(f"Failed to create ngrok tunnel: {str(e)}")
            print(f"Error creating ngrok tunnel: {str(e)}")
    else:
        st.error("NGROK_AUTH_TOKEN environment variable not found. Public URL will not be available.")
        print("Error: NGROK_AUTH_TOKEN environment variable not found")

if __name__ == "__main__":
    # Check if running in Colab
    import os
    is_colab = 'COLAB_GPU' in os.environ

    if is_colab:
        run_with_ngrok()

    main()

Writing ner_app2.py


In [ ]:
# ner_app3.py from google
%%writefile ner_app3.py
import streamlit as st
from typing import Dict, Union
from gliner import GLiNER
from pyngrok import ngrok
import pandas as pd
import os
import re
import torch  # Import torch

# Set page configuration
st.set_page_config(page_title="NER App", layout="wide")

# Function to merge adjacent entities of the same type
def merge_entities(entities):
    if not entities:
        return []
    merged = []
    current = entities[0]
    for next_entity in entities[1:]:
        if next_entity['entity'] == current['entity'] and (next_entity['start'] == current['end'] + 1 or next_entity['start'] == current['end']):
            current['word'] += ' ' + next_entity['word']
            current['end'] = next_entity['end']
        else:
            merged.append(current)
            current = next_entity
    merged.append(current)
    return merged

# Load the model (using GPU if available)
@st.cache_resource
def load_model():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    return GLiNER.from_pretrained("knowledgator/gliner-multitask-v1.0").to(device)

# Process text through the NER model
def process_text(text, labels, threshold, nested_ner):
    model = load_model()
    labels = [label.strip() for label in labels.split(",")]
    try:
        entities = [
            {
                "entity": entity["label"],
                "word": entity["text"],
                "start": entity["start"],
                "end": entity["end"],
                "score": entity["score"],
            }
            for entity in model.predict_entities(
                text, labels, flat_ner=not nested_ner, threshold=threshold
            )
        ]
        return merge_entities(entities)
    except Exception as e:
        st.error(f"Error during model prediction: {e}")
        return []

# Improved function to safely highlight entities in text
def highlight_entities(text, entities):
    # Sort entities by start position in descending order to avoid offset issues
    sorted_entities = sorted(entities, key=lambda x: x['start'], reverse=True)
    chars = list(text)
    for entity in sorted_entities:
        start = entity['start']
        end = entity['end']
        entity_type = entity['entity']
        color_hash = hash(entity_type) % 360
        bgcolor = f"hsl({color_hash}, 70%, 80%)"
        open_tag = f'<span style="background-color: {bgcolor}; padding: 1px 3px; border-radius: 3px;" title="{entity_type}">'
        close_tag = '</span>'
        chars.insert(end, close_tag)
        chars.insert(start, open_tag)
    highlighted_text = ''.join(chars).replace('\n', '<br>')
    return f'<div style="line-height: 1.6; white-space: pre-wrap; font-family: sans-serif;">{highlighted_text}</div>'

# Sample text for example
sample_text = """I recently purchased the Sony WH-1000XM4 Wireless Noise-Canceling Headphones from Amazon and I must say, I'm thoroughly impressed. The package arrived in New York within 2 days, thanks to Amazon Prime's expedited shipping.

The headphones themselves are remarkable. The noise-canceling feature works like a charm in the bustling city environment, and the 30-hour battery life means I don't have to charge them every day. Connecting them to my Samsung Galaxy S21 was a breeze, and the sound quality is second to none.

I also appreciated the customer service from Amazon when I had a question about the warranty. They responded within an hour and provided all the information I needed.

However, the headphones did not come with a hard case, which was listed in the product description. I contacted Amazon, and they offered a 10% discount on my next purchase as an apology.

Overall, I'd give these headphones a 4.5/5 rating and highly recommend them to anyone looking for top-notch quality in both product and service."""

sample_labels = "product, brand, location, features, rating"

# Main app function
def main():
    st.title("Named Entity Recognition App")
    st.write("Identify custom entities in your text using the GLiNER model")
    st.sidebar.header("Model Parameters")
    threshold = st.sidebar.slider("Confidence Threshold", 0.0, 1.0, 0.3, 0.01)
    nested_ner = st.sidebar.checkbox("Enable Nested NER", value=False)
    text_input = st.text_area("Enter your text:", height=200, value=sample_text)
    labels_input = st.text_input("Entity types (comma separated):", value=sample_labels)
    if st.button("Identify Entities"):
        if not text_input or not labels_input:
            st.error("Please enter both text and entity types")
            return
        with st.spinner("Processing..."):
            entities = process_text(text_input, labels_input, threshold, nested_ner)
            if not entities:
                st.info("No entities found with current settings. Try lowering the threshold.")
            else:
                st.subheader("Identified Entities")
                html_output = highlight_entities(text_input, entities)
                st.markdown(html_output, unsafe_allow_html=True)
                st.subheader("Entity Details")
                entity_df = pd.DataFrame(entities)
                if 'score' in entity_df.columns:
                    entity_df['score'] = entity_df['score'].round(3)
                st.dataframe(entity_df)
                st.subheader("Download Results")
                csv = entity_df.to_csv(index=False)
                st.download_button(
                    label="Download Entities as CSV",
                    data=csv,
                    file_name="entities.csv",
                    mime="text/csv",
                )
                annotated_text = text_input
                for entity in sorted(entities, key=lambda x: x['start'], reverse=True):
                    start = entity['start']
                    end = entity['end']
                    entity_type = entity['entity']
                    annotated_text = annotated_text[:start] + f"[{annotated_text[start:end]}]({entity_type})" + annotated_text[end:]
                st.download_button(
                    label="Download Annotated Text",
                    data=annotated_text,
                    file_name="annotated_text.txt",
                    mime="text/plain",
                )

# For running in Colab with pyngrok
def run_with_ngrok():
    # Get ngrok auth token from environment variable
    ngrok_token = os.environ.get('NGROK_AUTH_TOKEN')
    if ngrok_token:
        ngrok.set_auth_token(ngrok_token)
        try:
            public_url = ngrok.connect(port=8501)
            st.success(f"App running at: {public_url}")
            print(f"Public URL: {public_url}")  # Print for Colab output
        except Exception as e:
            st.error(f"Failed to create ngrok tunnel: {str(e)}")
            print(f"Error creating ngrok tunnel: {str(e)}") # Print for Colab output
    else:
        st.error("NGROK_AUTH_TOKEN environment variable not found. Public URL will not be available.")
        print("Error: NGROK_AUTH_TOKEN environment variable not found")  #Print

if __name__ == "__main__":
    import os
    is_colab = 'COLAB_GPU' in os.environ
    if is_colab:
        run_with_ngrok()
    main()

In [41]:
# # First, write the app code to a file
# %%writefile ner_app.py
# # [The entire code from the artifact above will be written to the file]

# # Install required packages
# !pip install streamlit pyngrok gliner -q

# Get the ngrok auth token from userdata
from google.colab import userdata
ngrok_token = userdata.get('NGROK_AUTH_TOKEN')

# Kill any existing Streamlit processes
!pkill -f streamlit || true

# Start Streamlit with the token as an environment variable
!streamlit run ner_app2.py --server.port=8501 &

# Give Streamlit a moment to start
import time
time.sleep(7)

# Check if the app is running
!ps aux | grep streamlit

# Create and display the public URL manually if needed
from pyngrok import ngrok
# Configure ngrok with the token
ngrok.set_auth_token(ngrok_token)

# Kill any existing tunnels
ngrok.kill()

# Start a new tunnel
public_url = ngrok.connect(port=8501)
print("\n" + "="*50)
print(f"📢 Access your app at this URL: {public_url}")
print("="*50 + "\n")

^C



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.230.12.212:8501

  Stopping...
root       34157  0.0  0.0   6484  2172 ?        S    21:39   0:00 grep streamlit


PyngrokNgrokHTTPError: ngrok client exception, API returned 400: {"error_code":102,"status_code":400,"msg":"invalid tunnel configuration","details":{"err":"yaml: unmarshal errors:\n  line 1: field port not found in type config.HTTPv2Tunnel"}}


In [ ]:
from google.colab import userdata
from pyngrok import ngrok
import subprocess
import time
import sys
import logging
import requests

# Configure logging (helps with debugging)
logging.basicConfig(level=logging.INFO)

# 1. Get ngrok Auth Token
ngrok_token = userdata.get('NGROK_AUTH_TOKEN')
if not ngrok_token:
    print("ERROR: NGROK_AUTH_TOKEN not found in userdata.  Please set it.", file=sys.stderr)
    sys.exit(1)

# 2. Kill Existing Processes (More Robustly)
def kill_processes():
    try:
        subprocess.run(["pkill", "-f", "streamlit"], check=True, capture_output=True)
        logging.info("Killed existing Streamlit processes.")
    except subprocess.CalledProcessError as e:
        logging.warning(f"Error killing Streamlit processes (likely none running): {e}")

    try:
        ngrok.kill()
        logging.info("Killed existing ngrok tunnels.")
    except Exception as e:
        logging.warning(f"Error killing ngrok tunnels: {e}")

kill_processes()  # Kill processes at the start


# 3. Start Streamlit (and Wait for it to Be Ready)
def start_streamlit(script_path, port=8501):
    """Starts Streamlit and waits for it to become accessible."""
    logging.info(f"Starting Streamlit on port {port}...")
    process = subprocess.Popen(
        ["streamlit", "run", script_path, f"--server.port={port}", "--server.headless=true"],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True  # Important for reading output as strings
    )

    # Wait for Streamlit to be ready (check for "running on" message)
    start_time = time.time()
    timeout = 60  # Maximum wait time in seconds
    while True:
        if time.time() - start_time > timeout:
            logging.error("Timeout: Streamlit did not start within the timeout period.")
            process.terminate()  # Kill the process if it times out.
            # Check for errors by reading the standard error of the process
            stderr_output = process.stderr.read()
            if stderr_output:
                logging.error(f"Streamlit stderr:\n{stderr_output}")
            return None

        output_line = process.stdout.readline() # read a line from stdout
        if output_line:
          logging.info(f"Streamlit Output: {output_line.strip()}")
        if "running on" in output_line.lower() or "External URL:" in output_line: # look for indication of successful startup
            logging.info("Streamlit is running.")
            break
        if process.poll() is not None: # check if streamlit process has exited,
                                      #None means it is still running
            logging.error("Streamlit process terminated unexpectedly.")
            stderr_output = process.stderr.read()
            if stderr_output:
                logging.error(f"Streamlit stderr:\n{stderr_output}")
            return None
        time.sleep(1)  # Check frequently, but don't spam

    return process

# 4.  Connect to ngrok (AFTER Streamlit is Ready)
def connect_ngrok(port=8501):
    """Connects to ngrok and returns the public URL."""
    ngrok.set_auth_token(ngrok_token)
    try:
        public_url = ngrok.connect(port)
        logging.info(f"ngrok tunnel established at: {public_url}")
        return public_url
    except Exception as e:
        logging.error(f"Error connecting to ngrok: {e}")
        return None

# --- Main Execution ---

# Assuming your Streamlit app script is named 'ner_app2.py'
streamlit_process = start_streamlit("ner_app2.py")

if streamlit_process:
    public_url_info = connect_ngrok()
    if public_url_info:
      print("\n" + "="*50)
      print(f"📢 Access your app at this URL: {public_url_info}")
      print("="*50 + "\n")

    # Keep the Colab cell alive (important!)
    try:
        while True:
            time.sleep(60) #keep the colab kernel alive
            if streamlit_process.poll() is not None: # check if streamlit is still running
                logging.warning("Streamlit process terminated. Restarting...")
                kill_processes()
                streamlit_process = start_streamlit("ner_app2.py")
                if streamlit_process:
                    public_url_info = connect_ngrok()  # Reconnect ngrok
                    if public_url_info:
                        print("\n" + "=" * 50)
                        print(f"📢 Access your app at this URL: {public_url_info}")
                        print("=" * 50 + "\n")

    except KeyboardInterrupt:
        logging.info("Stopping Streamlit and ngrok...")
        kill_processes()
        logging.info("Exiting.")


📢 Access your app at this URL: NgrokTunnel: "https://f1ee-35-230-12-212.ngrok-free.app" -> "http://localhost:8501"



PRODUCT, BRAND, SPECIFICATION, COLOR, FEATURE, PRICE, PROBLEM, RESOLUTION, SIZE, MODEL_NUMBER

id,query
1,What's the price of Samsung Galaxy S23 Ultra 256GB in Phantom Black?
2,How to fix overheating issue in 75" Samsung QN900B Neo QLED 8K TV?
3,Compare storage capacity between Galaxy Z Flip5 512GB and Fold4 1TB
4,Does Samsung Bespoke Jet™ Vacuum come in Navy Blue color?
5,Troubleshoot screen flickering on Odyssey G7 32" gaming monitor
6,Best Samsung soundbar with Dolby Atmos under $500
7,How long is warranty for 55" Samsung Frame TV 2023 model?
8,Replace battery for Galaxy Watch5 Pro 45mm Titanium Gray
9,Specifications for Samsung Smart Monitor M8 in Warm White
10,Does Samsung 980 Pro 2TB SSD work with PS5?
11,Compare Galaxy Buds2 Pro noise cancellation vs AirPods Pro
12,Price drop alert for 85" Samsung QLED Q80C series
13,How to clean mold in Samsung BESPOKE 4-Door Flex refrigerator?
14,Repair cost for cracked screen on Galaxy Tab S9 Ultra 14.6"
15,Energy efficiency rating of Samsung WindFree™ AC AR9500T
16,Where to buy Samsung 980 PRO SSD 1TB with heatsink?
17,Does Samsung S95B OLED TV support HDR10+ Gaming?
18,How to enable 144Hz refresh rate on Odyssey Neo G9 57" monitor?
19,Best Samsung laptop with 16GB RAM under $1000
20,Troubleshoot WiFi issues on Galaxy Book3 Ultra
21,Compare colors available for Galaxy Z Fold5 vs S23 Ultra
22,Return policy for defective Samsung T7 Shield 4TB SSD
23,Maximum load capacity for Samsung Jet Bot™+ robot vacuum
24,How durable is Samsung Galaxy XCover6 Pro military-grade phone?
25,Installation cost for Samsung 3-Tone Bespoke stove
26,Does Samsung 990 Pro 4TB SSD come with cloning software?
27,How to replace filter in Samsung Cube Air Purifier AX46
28,Best Samsung monitor for photo editing with color accuracy
29,Repair Samsung Bespoke Washer error code 4E
30,Compare charging speed: 45W vs 25W Samsung adapters
31,Price match guarantee on Samsung HW-Q990C Soundbar
32,How to clean Samsung AirDresser water tank?
33,Minimum room size for Samsung 9500 BTU WindFree™ AC
34,Does Galaxy Watch6 Classic support blood pressure monitoring?
35,Troubleshoot delayed notifications on Galaxy S23 FE
36,Compare battery life: Galaxy Book3 Pro vs MacBook Air M2
37,How scratch-resistant is Samsung S95C OLED TV screen?
38,Replacement parts for Samsung Jet™ 90 Complete vacuum
39,Best Samsung SSD for 4K video editing
40,Install Samsung SmartThings on Windows 11
41,Compare cooling performance: Bespoke vs Family Hub refrigerators
42,How to factory reset Galaxy A54 5G with broken screen?
43,Does Samsung 27" ViewFinity S8 monitor have HDMI 2.1?
44,Water damage repair cost for Galaxy Buds2 Pro
45,Best color calibration settings for S95C 65" OLED
46,How durable is Samsung Galaxy Tab Active4 Pro?
47,Compare RAM options for Galaxy Book3 Ultra laptops
48,Troubleshoot motion detection on Samsung S23 Ultra camera
49,Replacement cost for Samsung Bespoke induction cooktop
50,Does Galaxy Z Flip5 support wireless charging?
51,How to enable 200MP mode on Galaxy S23 Ultra?
52,Compare thickness: Galaxy S23 vs iPhone 14 Pro
53,Best Samsung T7 Shield case for outdoor use
54,How to clean Samsung Refrigerator Ice Maker mold?
55,Maximum SD card size for Galaxy Book3 Pro 360
56,Does Samsung 980 Pro SSD need heatsink for PS5?
57,Repair estimate for dented Bespoke refrigerator door
58,Compare brightness levels: QN900B vs S95C TVs
59,How to update firmware on Samsung T7 Touch SSD
60,Best Samsung monitor for coding with eye comfort
61,Does Galaxy Watch6 have fall detection feature?
62,Troubleshoot Dolby Atmos on HW-Q800C Soundbar
63,Compare warranty periods across Samsung SSD models
64,How to remove scratches from Galaxy Z Flip5 hinge?
65,Best color options for Samsung Bespoke Jet™ vacuum
66,Does Samsung 49" Odyssey G9 support Picture-by-Picture?
67,Installation requirements for Samsung Wall Mount Oven
68,Compare fan speeds: Samsung Jet™ 75 vs 90 models
69,How to replace Samsung Bespoke refrigerator water filter
70,Best Samsung SSD for gaming laptop upgrades
71,Does Galaxy S23 Ultra support 8K video recording?
72,Troubleshoot uneven cooling in Samsung WindFree™ AC
73,Compare weight: Galaxy Book3 Pro vs Surface Laptop 5
74,How to enable Game Mode on Samsung S95B OLED TV
75,Best Samsung monitor arm for 49" Odyssey G9
76,Does Galaxy Buds2 Pro have multipoint connectivity?
77,Compare thickness: QN900B vs LG G3 OLED TVs
78,How to clean Samsung AirDresser lint filter?
79,Replacement cost for Galaxy Watch6 Classic rotating bezel
80,Best Samsung SSD enclosure for 990 Pro
81,Does Galaxy Z Fold5 support S Pen Pro?
82,Troubleshoot Alexa integration on Samsung Frame TV
83,Compare charging speeds: 25W vs 45W Samsung adapters
84,How to enable Secure Folder on Galaxy S23 FE?
85,Best color profile settings for S95C 77" OLED
86,Does Samsung Bespoke Jet™ work on thick carpets?
87,Installation guide for Samsung Slide-in Gas Range
88,Compare noise levels: Jet 90 vs Dyson V15 vacuums
89,How to reset Samsung Bespoke Washer error code 5E
90,Best protective case for Galaxy Z Flip5
91,Does Galaxy Book3 Ultra have Thunderbolt 4 ports?
92,Troubleshoot slow charging on Galaxy S23 Ultra
93,Compare cooling systems: Odyssey G7 vs G9 monitors
94,How to enable Eye Comfort Mode on Galaxy Tab S9
95,Best Samsung T7 Shield configuration for photographers
96,Does Galaxy Watch6 Classic have ECG functionality?
97,Compare water resistance: Galaxy S23 vs S23 Ultra
98,How to clean Samsung Bespoke Refrigerator condenser coils
99,Best Samsung monitor for video conferencing
100,Does Galaxy Buds2 Pro support spatial audio?

In [2]:
import pandas as pd
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification

def extract_attributes(queries_csv, output_csv):
    """
    Extracts key attributes from a CSV file of user queries using a transformer-based NER model.

    Args:
        queries_csv: Path to the input CSV file containing a 'query' column.
        output_csv: Path to the output CSV file where the results will be saved.
    """

    # --- 1. Load the data ---
    try:
        df = pd.read_csv(queries_csv)
        if 'query' not in df.columns:
            raise ValueError("The input CSV must contain a 'query' column.")
    except FileNotFoundError:
        print(f"Error: The file '{queries_csv}' was not found.")
        return
    except pd.errors.ParserError:
        print(f"Error: Could not parse the CSV file '{queries_csv}'.  Check for formatting issues.")
        return
    except ValueError as ve:
        print(f"Error: {ve}")
        return


    # --- 2. Initialize the NER pipeline ---
    # Using a pre-trained model fine-tuned for product-related entities.  'dslim/bert-base-NER' is a good general NER model.
    # For more specific, product-focused NER, you might need to fine-tune a model yourself on a dataset of product descriptions/queries.
    try:
      #  tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")  #Or any suitable model
      #  model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
      #  nlp = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

        nlp = pipeline("ner", model="dslim/bert-base-NER", aggregation_strategy="simple") #Faster and handles some edge cases


    except Exception as e:
        print(f"Error loading the NER model: {e}.  Check your Hugging Face Transformers installation and model name.")
        return

    # --- 3. Define entity categories (Adapt based on your needs) ---
    #   This is crucial.  Define what constitutes a relevant entity for *your* attributes.
    entity_categories = {
        "PRODUCT": [],  # e.g., "laptop", "headphones", "smart TV"
        "BRAND": [],  # e.g., "Samsung", "Apple", "Sony"
        "SPECIFICATION": [],  # e.g., "16GB RAM", "4K resolution", "wireless"
        "COLOR": [],  # e.g. "black", "silver", "red"
        "FEATURE": [],  # e.g., "noise-canceling", "waterproof", "voice control"
        "PRICE": [],  # e.g., "$500", "under $1000", "cheap"  (Might need further processing)
        "PROBLEM": [],  #e.g. "broken", "defective", "malfunctioning"
        "RESOLUTION": [],  # e.g., "4K", "1080p", "HD"
        "SIZE": []       # e.g., "55-inch", "13-inch screen", "small"
    }


    # --- 4. Process each query ---
    for index, row in df.iterrows():
        query = row['query']
        try:
            ner_results = nlp(query)
        except Exception as e:
            print(f"Error processing query '{query}': {e}")
            continue  # Skip to the next query

        # --- 5. Extract and categorize entities ---
        extracted_entities = {category: [] for category in entity_categories}

        for entity in ner_results:
            entity_text = entity['word']
            entity_group = entity['entity_group']

            # Add more specific logic here, based on your entity categories and 'entity_group' from the model
            # This is where you map the model's output to YOUR desired attributes.
            if entity_group == "ORG" :  # Example: Treat organizations as brands
                extracted_entities["BRAND"].append(entity_text)
            elif entity_group == "MISC":   # Example: Potentially product names, specifications
                if any(keyword in entity_text.lower() for keyword in ["tv", "laptop", "phone", "headphones"]):
                    extracted_entities["PRODUCT"].append(entity_text)
                elif any(keyword in entity_text.lower() for keyword in ["inch", "gb", "ram"]):
                     extracted_entities["SPECIFICATION"].append(entity_text)

            elif entity_group == "PER":
              if any(keyword in entity_text.lower() for keyword in ["tv", "laptop", "phone", "headphones"]):
                    extracted_entities["PRODUCT"].append(entity_text)


            # Add more rules as needed, based on experimentation.  This is the most crucial part!

        # --- 6. Add extracted entities to the DataFrame ---
        for category, values in extracted_entities.items():
            # Join multiple entities of the same category with a comma.  Handle empty lists gracefully.
            df.loc[index, category] = ", ".join(values) if values else ""


    # --- 7. Save the modified DataFrame to a new CSV file ---
    try:
        df.to_csv(output_csv, index=False)
        print(f"Successfully processed queries and saved results to '{output_csv}'")
    except Exception as e:
        print(f"Error saving the output CSV: {e}")



# --- Example Usage ---
# Create a dummy CSV file for testing
data = {
    'query': [
        "I want a Samsung 55-inch 4K smart TV.",
        "Are there any noise-canceling headphones under $100?",
        "My iPhone 13 screen is broken.",
        "Looking for a cheap laptop with 16GB RAM.",
        "Do you have the new Sony WH-1000XM5 headphones?",
        "I need a black wireless mouse.",
        "What's the best gaming laptop?",
        "Is the Samsung Galaxy S23 waterproof?",
        "My Dell XPS 15 is malfunctioning.",
        "Show me 4k tv with Dolby vision",
        "I want a black colored TV",
        "which is the best tv under $2000"
    ]
}
queries_df = pd.DataFrame(data)
queries_df.to_csv('queries.csv', index=False)


# Run the attribute extraction
extract_attributes('queries.csv', 'output.csv')

# --- Inspect the output ---
output_df = pd.read_csv('output.csv')
print(output_df)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


Successfully processed queries and saved results to 'output.csv'
                                                query    PRODUCT    BRAND  \
0               I want a Samsung 55-inch 4K smart TV.        NaN  Samsung   
1   Are there any noise-canceling headphones under...        NaN      NaN   
2                      My iPhone 13 screen is broken.  iPhone 13      NaN   
3           Looking for a cheap laptop with 16GB RAM.        NaN      NaN   
4     Do you have the new Sony WH-1000XM5 headphones?        NaN      NaN   
5                      I need a black wireless mouse.        NaN      NaN   
6                      What's the best gaming laptop?        NaN      NaN   
7               Is the Samsung Galaxy S23 waterproof?        NaN      NaN   
8                   My Dell XPS 15 is malfunctioning.        NaN      NaN   
9                     Show me 4k tv with Dolby vision        NaN      NaN   
10                          I want a black colored TV        NaN      NaN   
11         

In [3]:
import pandas as pd
from transformers import pipeline
import re  # Import the regular expression module

def extract_attributes(queries_csv, output_csv):
    """
    Extracts key attributes from a CSV file of user queries using a transformer-based NER model
    and significantly improved, rule-based post-processing.
    """

    try:
        df = pd.read_csv(queries_csv)
        if 'query' not in df.columns:
            raise ValueError("The input CSV must contain a 'query' column.")
    except (FileNotFoundError, pd.errors.ParserError, ValueError) as e:
        print(f"Error loading or parsing CSV: {e}")
        return

    # --- 1. NER Pipeline (Still using dslim/bert-base-NER as a starting point) ---
    try:
        nlp = pipeline("ner", model="dslim/bert-base-NER", aggregation_strategy="simple")
    except Exception as e:
        print(f"Error loading NER model: {e}")
        return

    # --- 2. Improved Entity Categories (More Granular) ---
    entity_categories = {
        "PRODUCT": [],
        "BRAND": [],
        "SPECIFICATION": [],  # Expanded to handle more types
        "COLOR": [],
        "FEATURE": [],
        "PRICE": [],
        "PROBLEM": [],
        "RESOLUTION": [],
        "SIZE": [],
        "MODEL_NUMBER": []  # New category for model numbers (e.g., WH-1000XM5)
    }

    # --- 3. Keyword Lists (Crucial for Post-Processing) ---
    #  These lists are *essential* for improving accuracy.  Expand these lists!
    product_keywords = ["tv", "television", "laptop", "headphones", "phone", "smartphone", "tablet", "mouse", "keyboard", "monitor", "speaker", "camera", "printer", "router", "watch", "smartwatch", "earbuds"]
    feature_keywords = ["noise-canceling", "wireless", "bluetooth", "waterproof", "voice control", "4k", "hdr", "oled", "qled", "touchscreen", "gaming", "portable", "dolby vision", "dolby atmos"]
    problem_keywords = ["broken", "defective", "malfunctioning", "error", "issue", "problem", "not working", "doesn't work", "repair", "warranty"]
    resolution_keywords = ["4k", "1080p", "720p", "hd", "full hd", "ultra hd"]
    size_keywords = ["inch", "-inch", "''"]  # Include variations
    color_keywords = ["black", "white", "silver", "gray", "red", "blue", "green", "gold", "pink"]


    # --- 4. Enhanced Processing Loop ---
    for index, row in df.iterrows():
        query = row['query']
        try:
            ner_results = nlp(query)
        except Exception as e:
            print(f"Error processing query '{query}': {e}")
            continue

        extracted_entities = {category: [] for category in entity_categories}
        # Keep track of all entities found by the NER model, for use in rule-based extraction
        all_ner_entities = [entity['word'] for entity in ner_results]


        # --- 5. Rule-Based Extraction (The Key Improvement) ---

        # 5.1. PRODUCT
        for keyword in product_keywords:
            if keyword in query.lower():  # Case-insensitive check
                #  Check if the keyword is part of a larger word (e.g., "headphones" in "over-ear headphones")
                match = re.search(r'\b' + re.escape(keyword) + r'\b', query.lower()) # Use regex for whole word
                if match:
                    # Extract the full product name (if possible)
                    product_name = extract_full_product_name(query, match.start())
                    extracted_entities["PRODUCT"].append(product_name)

        # 5.2. BRAND
        for entity in ner_results:
            if entity['entity_group'] == 'ORG':
                extracted_entities["BRAND"].append(entity['word'])

        # 5.3. SPECIFICATION, RESOLUTION, SIZE (Combined and Improved)
        # Use regular expressions to find specifications, resolutions, and sizes
        spec_matches = re.findall(r'\b(\d+(?:\.\d+)?(?:gb|mb|tb|inch|hz))\b', query.lower())  #e.g. 16GB, 256gb, 5.5inch, 60hz
        for spec in spec_matches:
            extracted_entities["SPECIFICATION"].append(spec)

        for keyword in resolution_keywords:
            if keyword in query.lower():
               extracted_entities["RESOLUTION"].append(keyword)

        for keyword in size_keywords:
            size_match = re.search(r'(\d+(?:\.\d+)?)\s*' + re.escape(keyword), query.lower()) # Handles "55 inch" and "55-inch"
            if size_match:
                extracted_entities["SIZE"].append(size_match.group(0))  # Add the whole match (e.g., "55 inch")


        # 5.4. FEATURE
        for keyword in feature_keywords:
            if keyword in query.lower():
                extracted_entities["FEATURE"].append(keyword)

        # 5.5. PRICE (Regex-Based)
        price_match = re.search(r'(\$?\d+(?:,\d{3})*(?:\.\d{2})?)', query)  # Matches $, optional commas, and cents
        if price_match:
            price_text = price_match.group(1)
            # Check for "under", "over", etc.
            if "under" in query.lower() or "less than" in query.lower():
                price_text = "under " + price_text
            elif "over" in query.lower() or "more than" in query.lower():
                price_text = "over " + price_text
            extracted_entities["PRICE"].append(price_text)


        # 5.6. PROBLEM
        for keyword in problem_keywords:
            if keyword in query.lower():
                extracted_entities["PROBLEM"].append(keyword)

        # 5.7. COLOR
        for keyword in color_keywords:
             if keyword in query.lower():
                extracted_entities["COLOR"].append(keyword)

        #5.8 Model Number (using Regex)
        model_match = re.search(r'\b([A-Z0-9]{2,}-[A-Z0-9]{2,})\b', query)  # Example: WH-1000XM5
        if model_match:
              extracted_entities["MODEL_NUMBER"].append(model_match.group(1))


        # --- 6. Add to DataFrame ---
        for category, values in extracted_entities.items():
            df.loc[index, category] = ", ".join(list(dict.fromkeys(values))) if values else ""  # Remove duplicates and join


    try:
        df.to_csv(output_csv, index=False)
        print(f"Successfully processed queries and saved results to '{output_csv}'")
    except Exception as e:
        print(f"Error saving output CSV: {e}")


def extract_full_product_name(query, keyword_start_index):
    """
    Attempts to extract a more complete product name, going beyond just the keyword.
    This is a heuristic and may need further refinement.
    """
    # Expand to the left and right of the keyword, stopping at punctuation or the start/end of the string.
    start = keyword_start_index
    while start > 0 and query[start - 1].isalnum() or query[start-1] == ' ':
        start -= 1

    end = keyword_start_index
    while end < len(query) and query[end].isalnum() or query[end] == ' ':
        end += 1

    return query[start:end].strip()


# --- Example Usage (Using the same dummy data) ---
data = {
    'query': [
        "I want a Samsung 55-inch 4K smart TV.",
        "Are there any noise-canceling headphones under $100?",
        "My iPhone 13 screen is broken.",
        "Looking for a cheap laptop with 16GB RAM.",
        "Do you have the new Sony WH-1000XM5 headphones?",
        "I need a black wireless mouse.",
        "What's the best gaming laptop?",
        "Is the Samsung Galaxy S23 waterproof?",
        "My Dell XPS 15 is malfunctioning.",
        "Show me 4k tv with Dolby vision",
        "I want a black colored TV",
        "which is the best tv under $2000"
    ]
}
queries_df = pd.DataFrame(data)
queries_df.to_csv('queries.csv', index=False)

extract_attributes('queries.csv', 'output.csv')
output_df = pd.read_csv('output.csv')
print(output_df)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


IndexError: string index out of range

In [4]:
import pandas as pd
from transformers import pipeline
import re  # Import the regular expression module

def extract_attributes(queries_csv, output_csv):
    """
    Extracts key attributes from a CSV file of user queries using a transformer-based NER model
    and significantly improved, rule-based post-processing.
    """

    try:
        df = pd.read_csv(queries_csv)
        if 'query' not in df.columns:
            raise ValueError("The input CSV must contain a 'query' column.")
    except (FileNotFoundError, pd.errors.ParserError, ValueError) as e:
        print(f"Error loading or parsing CSV: {e}")
        return

    # --- 1. NER Pipeline (Still using dslim/bert-base-NER as a starting point) ---
    try:
        nlp = pipeline("ner", model="dslim/bert-base-NER", aggregation_strategy="simple")
    except Exception as e:
        print(f"Error loading NER model: {e}")
        return

    # --- 2. Improved Entity Categories (More Granular) ---
    entity_categories = {
        "PRODUCT": [],
        "BRAND": [],
        "SPECIFICATION": [],  # Expanded to handle more types
        "COLOR": [],
        "FEATURE": [],
        "PRICE": [],
        "PROBLEM": [],
        "RESOLUTION": [],
        "SIZE": [],
        "MODEL_NUMBER": []  # New category for model numbers (e.g., WH-1000XM5)
    }

    # --- 3. Keyword Lists (Crucial for Post-Processing) ---
    #  These lists are *essential* for improving accuracy.  Expand these lists!
    product_keywords = ["tv", "television", "laptop", "headphones", "phone", "smartphone", "tablet", "mouse", "keyboard", "monitor", "speaker", "camera", "printer", "router", "watch", "smartwatch", "earbuds"]
    feature_keywords = ["noise-canceling", "wireless", "bluetooth", "waterproof", "voice control", "4k", "hdr", "oled", "qled", "touchscreen", "gaming", "portable", "dolby vision", "dolby atmos"]
    problem_keywords = ["broken", "defective", "malfunctioning", "error", "issue", "problem", "not working", "doesn't work", "repair", "warranty"]
    resolution_keywords = ["4k", "1080p", "720p", "hd", "full hd", "ultra hd"]
    size_keywords = ["inch", "-inch", "''"]  # Include variations
    color_keywords = ["black", "white", "silver", "gray", "red", "blue", "green", "gold", "pink"]


    # --- 4. Enhanced Processing Loop ---
    for index, row in df.iterrows():
        query = row['query']
        try:
            ner_results = nlp(query)
        except Exception as e:
            print(f"Error processing query '{query}': {e}")
            continue

        extracted_entities = {category: [] for category in entity_categories}
        # Keep track of all entities found by the NER model, for use in rule-based extraction
        all_ner_entities = [entity['word'] for entity in ner_results]


        # --- 5. Rule-Based Extraction (The Key Improvement) ---

        # 5.1. PRODUCT
        for keyword in product_keywords:
            if keyword in query.lower():  # Case-insensitive check
                #  Check if the keyword is part of a larger word (e.g., "headphones" in "over-ear headphones")
                match = re.search(r'\b' + re.escape(keyword) + r'\b', query.lower()) # Use regex for whole word
                if match:
                    # Extract the full product name (if possible)
                    product_name = extract_full_product_name(query, match.start())
                    extracted_entities["PRODUCT"].append(product_name)

        # 5.2. BRAND
        for entity in ner_results:
            if entity['entity_group'] == 'ORG':
                extracted_entities["BRAND"].append(entity['word'])

        # 5.3. SPECIFICATION, RESOLUTION, SIZE (Combined and Improved)
        # Use regular expressions to find specifications, resolutions, and sizes
        spec_matches = re.findall(r'\b(\d+(?:\.\d+)?(?:gb|mb|tb|inch|hz))\b', query.lower())  #e.g. 16GB, 256gb, 5.5inch, 60hz
        for spec in spec_matches:
            extracted_entities["SPECIFICATION"].append(spec)

        for keyword in resolution_keywords:
            if keyword in query.lower():
               extracted_entities["RESOLUTION"].append(keyword)

        for keyword in size_keywords:
            size_match = re.search(r'(\d+(?:\.\d+)?)\s*' + re.escape(keyword), query.lower()) # Handles "55 inch" and "55-inch"
            if size_match:
                extracted_entities["SIZE"].append(size_match.group(0))  # Add the whole match (e.g., "55 inch")


        # 5.4. FEATURE
        for keyword in feature_keywords:
            if keyword in query.lower():
                extracted_entities["FEATURE"].append(keyword)

        # 5.5. PRICE (Regex-Based)
        price_match = re.search(r'(\$?\d+(?:,\d{3})*(?:\.\d{2})?)', query)  # Matches $, optional commas, and cents
        if price_match:
            price_text = price_match.group(1)
            # Check for "under", "over", etc.
            if "under" in query.lower() or "less than" in query.lower():
                price_text = "under " + price_text
            elif "over" in query.lower() or "more than" in query.lower():
                price_text = "over " + price_text
            extracted_entities["PRICE"].append(price_text)


        # 5.6. PROBLEM
        for keyword in problem_keywords:
            if keyword in query.lower():
                extracted_entities["PROBLEM"].append(keyword)

        # 5.7. COLOR
        for keyword in color_keywords:
             if keyword in query.lower():
                extracted_entities["COLOR"].append(keyword)

        #5.8 Model Number (using Regex)
        model_match = re.search(r'\b([A-Z0-9]{2,}-[A-Z0-9]{2,})\b', query)  # Example: WH-1000XM5
        if model_match:
              extracted_entities["MODEL_NUMBER"].append(model_match.group(1))


        # --- 6. Add to DataFrame ---
        for category, values in extracted_entities.items():
            df.loc[index, category] = ", ".join(list(dict.fromkeys(values))) if values else ""  # Remove duplicates and join


    try:
        df.to_csv(output_csv, index=False)
        print(f"Successfully processed queries and saved results to '{output_csv}'")
    except Exception as e:
        print(f"Error saving output CSV: {e}")


def extract_full_product_name(query, keyword_start_index):
    """
    Extracts a more complete product name, handling edge cases and variations.
    """
    # 1. Lowercase the query for case-insensitive matching.
    query = query.lower()
    keyword_start_index = query.find(query[keyword_start_index:].lower())


    # 2. Find the start of the potential product name (moving left).
    start = keyword_start_index
    while start > 0 and (query[start - 1].isalnum() or query[start - 1] in " -"):
        start -= 1

    # 3. Find the end of the potential product name (moving right).
    end = keyword_start_index
    # Find length of the matched word/keyword
    len_keyword = 0
    while end < len(query) and (query[end].isalnum() or query[end] in " -"):
        len_keyword = len_keyword + 1
        end = end + 1

    # 4. Extract the product name and clean it up.
    product_name = query[start : keyword_start_index + len_keyword]

     #Remove extra spaces using a regular expression:
    product_name = re.sub(r"\s+", " ", product_name).strip() # Replace multiple spaces with one

    return product_name



# --- Example Usage (Using the same dummy data + edge case test) ---
data = {
    'query': [
        "I want a Samsung 55-inch 4K smart TV.",
        "Are there any noise-canceling headphones under $100?",
        "My iPhone 13 screen is broken.",
        "Looking for a cheap laptop with 16GB RAM.",
        "Do you have the new Sony WH-1000XM5 headphones?",
        "I need a black wireless mouse.",
        "What's the best gaming laptop?",
        "Is the Samsung Galaxy S23 waterproof?",
        "My Dell XPS 15 is malfunctioning.",
        "Show me 4k tv with Dolby vision",
        "I want a black colored TV",
        "which is the best tv under $2000",
        "I want a tv"  # Edge case test
    ]
}
queries_df = pd.DataFrame(data)
queries_df.to_csv('queries.csv', index=False)

extract_attributes('queries.csv', 'output.csv')
output_df = pd.read_csv('output.csv')
print(output_df)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


Successfully processed queries and saved results to 'output.csv'
                                                query  \
0               I want a Samsung 55-inch 4K smart TV.   
1   Are there any noise-canceling headphones under...   
2                      My iPhone 13 screen is broken.   
3           Looking for a cheap laptop with 16GB RAM.   
4     Do you have the new Sony WH-1000XM5 headphones?   
5                      I need a black wireless mouse.   
6                      What's the best gaming laptop?   
7               Is the Samsung Galaxy S23 waterproof?   
8                   My Dell XPS 15 is malfunctioning.   
9                     Show me 4k tv with Dolby vision   
10                          I want a black colored TV   
11                   which is the best tv under $2000   
12                                        I want a tv   

                                           PRODUCT    BRAND SPECIFICATION  \
0             i want a samsung 55-inch 4k smart tv  Samsung

In [ ]:
import pandas as pd
from gliner import GLiNER
from typing import List

# Initialize GLiNER (requires torch)
model = GLiNER.from_pretrained("urchade/gliner_base")

# Define your entity categories with example prompts
entity_categories = {
    "PRODUCT": "electrical device or product name",
    "BRAND": "company or manufacturer name",
    "SPECIFICATION": "technical specification or measurement",
    "COLOR": "color description",
    "FEATURE": "product feature or capability",
    "PRICE": "price or monetary value",
    "PROBLEM": "product issue or defect",
    "RESOLUTION": "solution or resolution action",
    "SIZE": "size dimension",
    "MODEL_NUMBER": "alphanumeric product code"
}

def gliner_extract(text: str, threshold: float = 0.5) -> dict:
    entities = model.predict_entities(text, [v for v in entity_categories.values()], threshold=threshold)

    result = {k: [] for k in entity_categories.keys()}
    for entity in entities:
        for cat, prompt in entity_categories.items():
            if entity['label'] == prompt:
                result[cat].append(entity['text'])
    return {k: ", ".join(v) for k, v in result.items()}

def process_csv(input_path, output_path):
    df = pd.read_csv(input_path)
    df = df.join(df['query'].apply(lambda x: pd.Series(gliner_extract(x))))
    df.to_csv(output_path, index=False)

if __name__ == "__main__":
    process_csv("input_queries.csv", "output_entities.csv")

In [6]:
import pandas as pd

data = {
    'query': [
        # Samsung TVs
        "Is the Samsung QN90B 65-inch 4K Neo QLED TV available in black?",
        "I'm looking for a 55-inch Samsung TV with 4K resolution under $1000.",
        "Does the Samsung Frame TV come in 43-inch and have art mode?",
        "My Samsung Q80T 75-inch TV has a flickering screen.  What can I do?",
        "What's the price of a Samsung 8K QLED TV, specifically the QN900B?",
        "I need a curved Samsung TV, around 50 inches, with good sound.",
        "Is the Samsung S95B OLED TV good for gaming?",
        "Looking for a 32-inch Samsung smart TV, preferably white, for my kitchen.",
        "Does Samsung make any outdoor TVs with weatherproofing?",
        "My old Samsung TV remote is broken. Where can I get a replacement?",
        "Are there any Samsung TVs with built-in Alexa?",
        "I want a Samsung TV with Dolby Atmos and a 120Hz refresh rate.",
        "Compare the Samsung QN85B and QN90B 50-inch models.",
        "What's the best 75-inch Samsung TV for a bright room?",
        "Is there a Samsung TV that supports both FreeSync and G-Sync?",

        # Samsung Phones
        "Does the Samsung Galaxy S23 Ultra come in green and with 512GB storage?",
        "I want a Samsung phone with a good camera for under $800.",
        "My Samsung Galaxy Z Fold 4 screen is cracked. Is it covered by warranty?",
        "What's the battery life like on the Samsung Galaxy A54?",
        "Is the Samsung Galaxy S22 still a good phone in 2024?",
        "I need a rugged Samsung phone that's waterproof and dustproof.",
        "Does the Samsung Galaxy S23 have expandable storage?",
        "I dropped my Samsung phone and the screen is black.  Can it be repaired?",
        "What's the cheapest Samsung phone with 5G?",
        "Compare the camera on the Samsung Galaxy S23+ and the S23 Ultra.",
        "Is there a Samsung phone with a stylus?",
        "My Samsung Galaxy Note 20 is running slow. How can I speed it up?",
        "Which Samsung phone has the best zoom capabilities?",
        "Does the Samsung Galaxy Z Flip 5 have wireless charging?",
        "I want a small Samsung phone, something like the Galaxy S22.",

        # Samsung Headphones & Earbuds
        "Are the Samsung Galaxy Buds 2 Pro noise-canceling?",
        "I lost one of my Samsung Galaxy Buds. Can I buy a single replacement?",
        "What's the difference between the Samsung Galaxy Buds 2 and Buds Pro?",
        "Do the Samsung Galaxy Buds FE have good bass?",
        "Are Samsung earbuds compatible with iPhones?",
        "My Samsung headphones won't connect to my laptop.  What should I do?",
        "How long does the battery last on the Samsung Galaxy Buds Live?",
        "I need waterproof earbuds for swimming. Does Samsung make any?",
        "Are there any Samsung headphones with a built-in microphone?",
        "What's the best Samsung earbuds for working out?",

        # Samsung Tablets
        "Does the Samsung Galaxy Tab S9 Ultra have a 120Hz display?",
        "I want a Samsung tablet with an S Pen for under $500.",
        "My Samsung Galaxy Tab S8 screen is unresponsive. How do I fix it?",
        "Is the Samsung Galaxy Tab A8 good for reading ebooks?",
        "What's the largest screen size available for a Samsung tablet?",
        "I need a Samsung tablet with long battery life for travel.",
        "Does the Samsung Galaxy Tab S9 FE come in different colors?",
        "My Samsung tablet is frozen. How do I restart it?",
        "What's the best Samsung tablet for drawing and note-taking?",
        "Can I use a Samsung tablet as a second screen for my laptop?",
        "Is the Samsung Galaxy Tab Active Pro ruggedized?",

        # Samsung Watches
        "Does the Samsung Galaxy Watch 6 Classic have ECG and blood pressure monitoring?",
        "I want a Samsung watch with LTE connectivity.",
        "My Samsung Galaxy Watch 5 won't charge.  What's wrong?",
        "Is the Samsung Galaxy Watch 4 still a good buy?",
        "What's the battery life of the Samsung Galaxy Watch 6?",
        "I need a Samsung watch that's good for tracking fitness.",
        "Does the Samsung Galaxy Watch have fall detection?",
        "My Samsung watch band broke. Where can I get a new one?",
        "What's the best Samsung watch for sleep tracking?",
        "Can I make calls from a Samsung Galaxy Watch?",

        # Combinations & General Queries
        "Looking for a Samsung soundbar with Dolby Atmos and a wireless subwoofer.",
        "Do you have any Samsung appliances, like refrigerators or washing machines?",  #(Out of scope, but good to include for testing)
        "What are the latest Samsung product releases?",
        "I need a Samsung charger for my phone and watch.",
        "Where can I find the nearest Samsung service center?",
        "Are there any current promotions on Samsung products?",
        "I want to trade in my old Samsung phone for a new one.",
        "Do you sell refurbished Samsung products?",
        "What's the warranty on Samsung products?",
        "I'm having trouble setting up my Samsung SmartThings hub.",
        "Can I control my Samsung TV with my phone?",
        "I want a Samsung monitor for gaming with a high refresh rate.",
        "Does Samsung offer student discounts?",

        # More specific combinations
        "Looking for a black Samsung Galaxy S23 Ultra with 256GB and the S Pen.",
        "Need a 65-inch Samsung QLED TV with 4K resolution and HDMI 2.1.",
        "My Samsung Galaxy Buds Pro case is lost; can I buy a replacement?",
        "Is there a silver Samsung Galaxy Watch 6 with a rotating bezel?",
        "Want a Samsung Galaxy Tab S9+ with 5G and the keyboard cover.",
        "Looking for a Samsung phone with a really good camera, over 50MP.",
        "Need a rugged Samsung tablet that can withstand drops and water.",
        "I want a Samsung TV that's specifically designed for gaming.",
        "Does Samsung make any noise-canceling headphones that are also wireless?",
        "My Samsung phone screen is cracked, but it's still under warranty.",
        "Is there a Samsung watch that can track my swimming workouts?",
        "Looking for a large Samsung tablet, at least 12 inches, for drawing.",
        "I want a Samsung phone that has a really long battery life, over 4000mAh.",
        "Does Samsung have any TVs with built-in voice assistants like Bixby?",
        "My Samsung earbuds keep disconnecting from my phone; is this a common problem?",

        # More edge cases / problem scenarios
        "My brand new Samsung QN95B TV has dead pixels. What should I do?",
        "The Samsung Galaxy Z Flip 4 hinge is making a creaking noise.",
        "I can't update the software on my Samsung Galaxy Watch 5.",
        "My Samsung tablet is stuck in a boot loop.",
        "The battery on my Samsung phone drains very quickly, even when I'm not using it.",
        "My Samsung TV remote is unresponsive, even with new batteries.",
        "I accidentally spilled water on my Samsung laptop. What should I do?",  #(Laptop is out of scope, but good for testing robustness)
        "The charging port on my Samsung phone is loose.",
        "My Samsung soundbar won't connect to my TV via Bluetooth.",
        "I forgot the password to my Samsung tablet.",
        "My Samsung Galaxy Buds Pro are causing ear irritation.",
        "The screen on my Samsung watch is scratched. Can it be replaced?",
        "I'm experiencing audio lag when using my Samsung headphones with my TV.",
        "My Samsung phone is overheating when I play games.",
        "The fingerprint sensor on my Samsung phone isn't working reliably.",

        # Asking for comparisons
        "What's the main difference between the Samsung Galaxy S23 and the Google Pixel 7?",  #(Comparison with another brand)
        "Compare the Samsung QN90B and the LG C2 OLED TVs." ,  #(Comparison with another brand)
        "Which is better for gaming, the Samsung Galaxy S23 Ultra or the iPhone 14 Pro Max?", #(Comparison with another brand)
        "Should I get the Samsung Galaxy Buds 2 Pro or the Apple AirPods Pro 2?", #(Comparison with another brand)
        "What are the pros and cons of the Samsung Galaxy Watch 6 vs. the Watch 5?",
        "How does the Samsung Galaxy Tab S9 compare to the iPad Air?" #(Comparison with another brand)

    ]
}

queries_df = pd.DataFrame(data)
print(f"Number of queries generated: {len(queries_df)}")
queries_df.to_csv('samsung_queries_100.csv', index=False)

output_df = pd.read_csv('samsung_queries_100.csv')
print(output_df)

Number of queries generated: 110
                                                 query
0    Is the Samsung QN90B 65-inch 4K Neo QLED TV av...
1    I'm looking for a 55-inch Samsung TV with 4K r...
2    Does the Samsung Frame TV come in 43-inch and ...
3    My Samsung Q80T 75-inch TV has a flickering sc...
4    What's the price of a Samsung 8K QLED TV, spec...
..                                                 ...
105  Compare the Samsung QN90B and the LG C2 OLED TVs.
106  Which is better for gaming, the Samsung Galaxy...
107  Should I get the Samsung Galaxy Buds 2 Pro or ...
108  What are the pros and cons of the Samsung Gala...
109  How does the Samsung Galaxy Tab S9 compare to ...

[110 rows x 1 columns]


In [7]:
# 1. What is the best Samsung smartphone PRODUCT with FEATURE wireless charging under $500?
# 2. Compare Samsung Galaxy S23 MODEL_NUMBER specifications across different COLOR options
# 3. How to resolve PROBLEM of battery drain on Samsung NOTE PRODUCT
# 4. Samsung TV BRAND comparison by SIZE and PRICE range
# 5. What SPECIFICATION makes Samsung A53 MODEL_NUMBER superior to competitors?
# 6. Cheapest Samsung smartphone with FEATURE 5G and under $300 PRICE
# 7. Samsung refrigerator COLOR options for modern kitchen design
# 8. How to fix RESOLUTION issues with Samsung PRODUCT display
# 9. Best Samsung laptop with highest RAM SPECIFICATION
# 10. Samsung BRAND washing machine models with energy-efficient FEATURE
# 11. Compare Samsung smartphone PRICE across different MODEL_NUMBER variants
# 12. Which Samsung PRODUCT has the best camera SPECIFICATION?
# 13. Troubleshoot PROBLEM with Samsung tablet not turning on
# 14. Samsung smart TV SIZE recommendations for living rooms
# 15. Most affordable Samsung BRAND smartphone with premium FEATURE
# 16. Samsung Galaxy Z PRODUCT series COLOR availability
# 17. Detailed SPECIFICATION of Samsung S22 MODEL_NUMBER camera system
# 18. How to resolve RESOLUTION problems on Samsung monitor
# 19. Samsung laptop PRICE range for professional users
# 20. Best Samsung refrigerator FEATURE for large families
# 21. Samsung smartphone COLOR options for professional settings
# 22. What SPECIFICATION differentiates Samsung A-series MODEL_NUMBER?
# 23. Troubleshoot battery PROBLEM on Samsung Galaxy devices
# 24. Samsung TV BRAND reliability comparison
# 25. Most compact Samsung laptop with powerful SPECIFICATION
# 26. Samsung smartphone PRODUCT with longest battery life
# 27. Compare Samsung tablet PRICE across different MODEL_NUMBER
# 28. How to fix audio RESOLUTION on Samsung smart TV
# 29. Samsung smartphone FEATURE comparison for photography enthusiasts
# 30. Best Samsung BRAND appliance for small apartments
# 31. Samsung Galaxy Z PRODUCT folding mechanism SPECIFICATION
# 32. Affordable Samsung smartphone COLOR options
# 33. Troubleshoot PROBLEM with Samsung wireless earbuds
# 34. Samsung monitor SIZE recommendations for graphic designers
# 35. Which Samsung PRODUCT offers best value for money?
# 36. Samsung smartphone MODEL_NUMBER with water resistance FEATURE
# 37. Compare Samsung TV PRICE across different SIZE ranges
# 38. How to resolve display RESOLUTION issues on Samsung devices
# 39. Samsung laptop SPECIFICATION for gaming performance
# 40. Best Samsung refrigerator COLOR for minimalist kitchens
# 41. Samsung Galaxy S PRODUCT line camera SPECIFICATION breakdown
# 42. Troubleshoot charging PROBLEM on Samsung smartphones
# 43. Samsung smart TV FEATURE comparison for streaming
# 44. Most lightweight Samsung laptop MODEL_NUMBER
# 45. Samsung smartphone BRAND market positioning analysis
# 46. Compare Samsung tablet SPECIFICATION across different models
# 47. How to fix connectivity RESOLUTION on Samsung devices
# 48. Samsung refrigerator PRICE range for budget buyers
# 49. Best Samsung PRODUCT for mobile photography
# 50. Samsung Galaxy NOTE COLOR options for professionals
# 51. Troubleshoot software PROBLEM on Samsung smart devices
# 52. Samsung monitor SPECIFICATION for video editing
# 53. Affordable Samsung smartphone FEATURE comparison
# 54. Compare Samsung TV MODEL_NUMBER across different PRICE points
# 55. How to resolve storage RESOLUTION on Samsung tablets
# 56. Samsung laptop BRAND performance benchmarks
# 57. Best Samsung refrigerator FEATURE for energy efficiency
# 58. Samsung smartphone PRODUCT durability SPECIFICATION
# 59. Troubleshoot Bluetooth PROBLEM on Samsung accessories
# 60. Samsung smart TV SIZE guide for different room dimensions
# 61. Compare Samsung Galaxy Z PRODUCT series SPECIFICATION
# 62. Samsung smartphone COLOR trends for 2024
# 63. How to fix camera RESOLUTION issues on Samsung devices
# 64. Samsung laptop PRICE comparison for students
# 65. Best Samsung BRAND wearable technology
# 66. Samsung refrigerator MODEL_NUMBER with smart home FEATURE
# 67. Troubleshoot battery charging PROBLEM on Samsung devices
# 68. Samsung monitor COLOR calibration guide
# 69. Compare Samsung smartphone SPECIFICATION across price ranges
# 70. How to resolve network RESOLUTION on Samsung tablets
# 71. Samsung TV PRODUCT lineup with best picture quality
# 72. Best Samsung laptop for professional SPECIFICATION
# 73. Troubleshoot audio PROBLEM on Samsung smart TV
# 74. Samsung smartphone COLOR psychology and design
# 75. Compare Samsung refrigerator SIZE for different kitchen layouts
# 76. How to fix software update RESOLUTION on Samsung devices
# 77. Samsung Galaxy A PRODUCT series budget SPECIFICATION
# 78. Best Samsung monitor FEATURE for remote work
# 79. Troubleshoot Wi-Fi PROBLEM on Samsung smart home devices
# 80. Samsung smartphone MODEL_NUMBER battery performance comparison
# 81. Compare Samsung TV BRAND reliability ratings
# 82. How to resolve display COLOR calibration issues
# 83. Samsung laptop SPECIFICATION for content creators
# 84. Best Samsung refrigerator with smart home FEATURE
# 85. Troubleshoot connectivity PROBLEM on Samsung wireless devices
# 86. Samsung smartphone PRODUCT camera low-light SPECIFICATION
# 87. Compare Samsung tablet SIZE for portability
# 88. How to fix charging PORT RESOLUTION on Samsung devices
# 89. Samsung smart TV PRICE range for budget buyers
# 90. Best Samsung laptop COLOR options for professionals
# 91. Troubleshoot software compatibility PROBLEM on Samsung devices
# 92. Samsung smartphone FEATURE comparison for battery life
# 93. Compare Samsung monitor SPECIFICATION for gaming
# 94. How to resolve audio output RESOLUTION on Samsung TV
# 95. Samsung refrigerator MODEL_NUMBER energy efficiency ratings
# 96. Best Samsung PRODUCT for mobile gaming
# 97. Troubleshoot Bluetooth pairing PROBLEM on Samsung accessories
# 98. Samsung smartphone COLOR trends in professional market
# 99. Compare Samsung TV SIZE recommendations
# 100. How to resolve system performance RESOLUTION on Samsung devices

SyntaxError: invalid decimal literal (<ipython-input-7-7344f275c6a0>, line 6)